Why Handle Tool Errors?

When your LLM calls a tool (for example, to fetch data or call an API), many things can go wrong:

- API timeout or failure
- Invalid input from the LLM
- Empty database result
- Network or permission error

If you don’t handle these errors, your agent will crash or get stuck, breaking the conversation flow. So, you need to gracefully handle exceptions and return safe messages or structured error responses.

## How to handle tool errors


In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [3]:
# Define tool
from langchain_core.tools import tool

@tool
def complex_tool(int_arg: int, float_arg: float, dict_arg: dict) -> int:
    """Do something complex with a complex tool."""
    return int_arg * float_arg


llm_with_tools = llm.bind_tools(
    [complex_tool],
)

# Define chain
chain = llm_with_tools | (lambda msg: msg.tool_calls[0]["args"]) | complex_tool

In [4]:
chain.invoke(
    "use complex tool. the args are 5, 2.1, empty dictionary. don't forget dict_arg"
)

10.5

## Try/except tool call
The simplest way to more gracefully handle errors is to try/except the tool-calling step and return a helpful message on errors:



In [73]:
from langchain_core.tools import tool

@tool
def divide_numbers(a: float, b: float) -> float:
    """Divide two numbers safely."""
    return a / b

def handle_division_errors(e: Exception) -> str:
    if isinstance(e, ZeroDivisionError):
        return "Error: Division by zero is not allowed. Please provide a non-zero denominator."
    elif isinstance(e, TypeError):
        return "Error: Both inputs must be numbers (int or float). Please check your input types."
    elif isinstance(e, ValueError):
        return "Error: Invalid input value. Please enter valid numeric values."
    else:
        return f"Unexpected error: {e}. Please try again."



In [75]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent,Tool


load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
tools = [
    Tool(
        name="Divide",
        func=divide_numbers,  # link our tool
        description="Use this tool to divide two numbers.",
        
    )
]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    verbose=True,  # shows step-by-step reasoning
)


In [65]:
response = agent.run("What is 8.0 divided by 2.0?")
print("Agent output:", response)



> Entering new AgentExecutor chain...
<think>
Okay, so I need to figure out what 8.0 divided by 2.0 is. Hmm, let me think about this. I remember that division is splitting a number into equal parts. So, if I have 8.0 and I divide it by 2.0, that means I'm splitting 8 into two equal parts. 

Wait, 8 divided by 2 is something I've done before. Isn't that 4? Yeah, because 2 times 4 is 8. So, 8.0 divided by 2.0 should be 4.0. But let me make sure I'm not making a mistake here.

Maybe I can think about it in another way. If I have 8 apples and I give them equally to 2 friends, each friend gets 4 apples. That makes sense. So, 8.0 divided by 2.0 is definitely 4.0. I don't think there's any other way around it. 

I could also use the division tool provided. Let me try that. So, I'll input 8.0 as the dividend and 2.0 as the divisor. Using the Divide tool, I get 4.0 as the result. Yep, that confirms what I thought earlier. 

I don't see any complications here because both numbers are positive 

In [71]:
response = agent.run("What is 8.0 divided by 0.0?")
print("Agent output:", response)



> Entering new AgentExecutor chain...
<think>
Okay, so I need to figure out what 8.0 divided by 0.0 is. Hmm, dividing by zero... I remember from math class that dividing by zero is undefined. But wait, how does that work with computers? I think sometimes they handle it by showing an error or something like infinity. Let me think about this step by step.

First, I know that in mathematics, any number divided by zero is undefined because you can't divide by nothing. It's like trying to split something into zero parts, which doesn't make sense. So theoretically, 8.0 divided by 0.0 should be undefined. But in programming, especially with floating-point numbers, dividing by zero often results in infinity or negative infinity, depending on the signs. Since both numbers here are positive, maybe it would result in positive infinity.

Wait, but the tool I'm using is called Divide, which probably expects a numerical result. I'm not sure how it handles division by zero. Maybe it returns an erro